# Training file (testing if training works)

In [1]:
import sys
sys.path.append('../')

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CTCLoss
import torch.optim as optim

from src.training.trainer import train_model
from src.dataset.custom_dataset import OdometerDataset
from src.dataset.base_dataset import base_collate_fn
from src.models.crnn import CRNN

/home/yannou/OneDrive/Documents/3_PRO/Box2Read/env-ocr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:

data_dir = '/home/yannou/OneDrive/Documents/3_PRO/carviz/data/ocr/odometer_reader/milestone_box'
labels_file = '/home/yannou/OneDrive/Documents/3_PRO/carviz/data/ocr/odometer_reader/milestone_box/milestone_labels.json'

# Définir les transformations
transform = transforms.Compose([
    transforms.RandomRotation(7),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

transform_val = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Initialiser le dataset
dataset = OdometerDataset(root_dir=data_dir, split="train", labels_file=labels_file, img_height=32, img_width=100, transform=transform)
dataset_val = OdometerDataset(root_dir=data_dir, split="val", labels_file=labels_file, img_height=32, img_width=100, transform=transform_val)


# Créer les DataLoaders
train_loader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=base_collate_fn)
valid_loader = DataLoader(dataset_val, batch_size=64, shuffle=False, collate_fn=base_collate_fn)



In [5]:
# Initialiser le modèle
num_class = len(OdometerDataset.LABEL2CHAR) + 1
crnn = CRNN(img_channel=1, img_height=32, img_width=100, num_class=num_class, model_size="n", leaky_relu=True).to(device)

# Définir les paramètres d'entraînement
lr = 0.001
epochs = 15
decode_method = 'beam_search'
beam_size = 10
label2char = OdometerDataset.LABEL2CHAR

In [6]:
print(f"Working on {device}")
# Appeler la fonction train_model
trained_model = train_model(
    model=crnn,
    train_loader=train_loader,
    valid_loader=valid_loader,
    label2char=label2char,
    device=device,
    lr=lr,
    epochs=epochs,
    decode_method=decode_method,
    beam_size=beam_size,
    criterion=CTCLoss(reduction='sum', zero_infinity=True).to(device),
    optimizer=optim.Adam(crnn.parameters(), lr=lr),
    project_name="odometer-reader",
    run_name="crnn-n", 
    checkpoint=2
)



Working on cuda:0


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yann-t (carviz-com). Use `wandb login --relogin` to force relogin


Epochs:   7%|▋         | 1/15 [01:25<20:02, 85.87s/it]

Epoch 1: train_loss=20.262676032982558, train_accuracy=0.0, val_loss=13.68884794590837, val_accuracy=0.0, train_word_accuracy=0.0, train_char_accuracy=0.0022771838825985198, train_average_levenshtein_distance=5.179705400981997val_word_accuracy=0.0, val_char_accuracy=0.0, val_average_levenshtein_distance=5.160365058670143


Epochs:  13%|█▎        | 2/15 [02:48<18:08, 83.74s/it]

...

Epochs:  93%|█████████▎| 14/15 [18:00<01:10, 70.36s/it]

Epoch 14: train_loss=4.645981064716845, train_accuracy=0.2500818330605565, val_loss=3.8914755929402296, val_accuracy=0.3076923076923077, train_word_accuracy=0.2500818330605565, train_char_accuracy=0.5426023151369473, train_average_levenshtein_distance=1.5590834697217677val_word_accuracy=0.3076923076923077, val_char_accuracy=0.6336533602829711, val_average_levenshtein_distance=1.2985658409387224


Epochs: 100%|██████████| 15/15 [19:08<00:00, 76.56s/it]

Epoch 15: train_loss=4.29250313966442, train_accuracy=0.27430441898527, val_loss=3.3745337813276666, val_accuracy=0.3898305084745763, train_word_accuracy=0.27430441898527, train_char_accuracy=0.5774558795622746, train_average_levenshtein_distance=1.448772504091653val_word_accuracy=0.3898305084745763, val_char_accuracy=0.6526023244062658, val_average_levenshtein_distance=1.1225554106910038


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train/average_levenshtein_distance,█████▇▆▅▄▃▂▂▁▁▁
train/char_accuracy,▁▁▁▁▁▁▂▂▃▅▆▆▇██
train/train_loss,█▅▅▅▅▄▄▃▂▂▂▂▁▁▁
train/word_accuracy,▁▁▁▁▁▁▁▁▂▃▄▅▆▇█
val/average_levenshtein_distance,█████▇▆▄▄▂▂▂▂▁▁
val/char_accuracy,▁▁▁▁▁▁▂▃▃▅▆▇▇██
val/val_loss,████▇▆▆▄▃▂▃▂▁▁▁
val/word_accuracy,▁▁▁▁▁▁▁▁▁▃▄▅▆▇█
epoch,15
train/average_levenshtein_distance,1.44877
